In [ ]:
import numpy as np # linear algebra
import pandas as pd
# pd.set_option("display.max_rows", 101)
import os
import cv2
import json
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.size"] = 15
import seaborn as sns
from collections import Counter
from PIL import Image
import math
import seaborn as sns
from collections import defaultdict
from pathlib import Path
import cv2
from tqdm import tqdm

In [ ]:
all_folds = os.listdir('../input/severstal-steel-defect-detection/train_images/')
all_folds = [x for x in all_folds]
train_df = pd.read_csv("../input/severstal-steel-defect-detection/train.csv")
train_df.info()

有缺陷、无缺陷图片数量

In [ ]:
image_defects = train_df['ImageId'].unique().tolist()
image_no_defects=[item for item in all_folds if item not in image_defects]
print("有缺陷的图片：{}".format(len(image_defects)))
print("没有缺陷的图片：{}".format(len(image_no_defects)))

各类别图片的数量

具有多标记图片的数量

In [ ]:
images_class_num = {'0':0,'1':0,'2':0,'3':0,'4':0}
images_class = {'0':[],'1':[],'2':[],'3':[],'4':[]}
images_mulitlabel = {}
images_mulitlabel_num = {}

#####
images_class_num['0'] = len(image_no_defects)
images_class_num['1'] = (train_df.ClassId == 1).sum()
images_class_num['2'] = (train_df.ClassId == 2).sum()
images_class_num['3'] = (train_df.ClassId == 3).sum()
images_class_num['4'] = (train_df.ClassId == 4).sum()

images_class['0'] = image_no_defects
images_class['1'] = [i for i in train_df.ImageId[train_df.ClassId == 1]]
images_class['2'] = [i for i in train_df.ImageId[train_df.ClassId == 2]]
images_class['3'] = [i for i in train_df.ImageId[train_df.ClassId == 3]]
images_class['4'] = [i for i in train_df.ImageId[train_df.ClassId == 4]]


for i in image_defects:
    classid = train_df.ClassId[train_df.ImageId==i]
    if len(classid)>1:
        classid = str(classid.tolist())
#         images_mulitlabel[i] = classid.tolist()
        if str(classid) not in images_mulitlabel.keys():
            images_mulitlabel[classid] = [i]
        else:
            images_mulitlabel[classid].append(i)

images_mulitlabel_num = images_mulitlabel.copy()
for i in images_mulitlabel_num:
    images_mulitlabel_num[i] = len(images_mulitlabel_num[i])

In [ ]:
# 有瑕疵、无瑕疵图片的数量

print(images_mulitlabel_num)
sns.barplot(x=[i for i in images_mulitlabel_num], y=[images_mulitlabel_num[i] for i in images_mulitlabel_num])
plt.show()

In [ ]:
# 具有多种瑕疵的图片的数量
# '[3, 4]'代表具有3、4标记的图片

print(images_mulitlabel_num)
sns.barplot(x=[i for i in images_mulitlabel_num], y=[images_mulitlabel_num[i] for i in images_mulitlabel_num])
plt.show()

In [ ]:
# 每种瑕疵类别的图片数量

print(images_class_num)
sns.barplot(x=[i for i in images_class_num], y=[images_class_num[i] for i in images_class_num])
plt.show()

查看图片

In [ ]:
palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]
train_path = "../input/severstal-steel-defect-detection/train_images"

def name_to_mask(name):
    idx = train_df[train_df.ImageId==name].index.tolist()
    mask = np.zeros((256, 1600, 4), dtype=np.uint8)
    if idx==[]:
        return mask
    
    # rle to mask
    for i in idx:
        mask_embedding = np.zeros(1600*256, dtype=np.uint8)
        rle = train_df.EncodedPixels[i].split(" ")
        classid = train_df.ClassId[i]
#         print("classid: {}".format(classid))
        positions = map(int, rle[0::2])
        length = map(int, rle[1::2])
        for pos, le in zip(positions, length):
            mask_embedding[pos-1:pos+le-1] = 1
        mask[:, :, classid-1] = mask_embedding.reshape(256, 1600, order='F')
    return mask

def show_mask_image(name):
    mask = name_to_mask(name)
    img = cv2.imread(str(train_path +"/"+ name))
    fig, ax = plt.subplots(figsize=(15, 15))
    for ch in range(4):
        contours, _ = cv2.findContours(mask[:, :, ch], cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for i in range(0, len(contours)):
            cv2.polylines(img, contours[i], True, palet[ch], 2)
    ax.set_title(name)
    ax.imshow(img)
    plt.show()    
    
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    ax[i].axis('off')
    ax[i].imshow(np.ones((50, 50, 3), dtype=np.uint8) * palet[i])
    ax[i].set_title("class color: {}".format(i+1))

fig.suptitle("each class colors")
plt.show()

In [ ]:
for name in images_class['0'][:3]:
    show_mask_image(name)

In [ ]:
for name in images_class['1'][:3]:
    show_mask_image(name)

In [ ]:
for name in images_class['2'][:3]:
    show_mask_image(name)

In [ ]:
for name in images_class['3'][:3]:
    show_mask_image(name)

In [ ]:
for name in images_class['4'][:3]:
    show_mask_image(name)

In [ ]:
for name in images_mulitlabel['[2, 3]'][:1]:
    show_mask_image(name)